Gradients, Edges, and Lineaments
---------------------------------------------------

Welcome back!

In this section, we're going to explore image gradients, edge detectors, and lineament analysis.  In other words, we're going to try to start detecting features in images that aren't defined by the raw value of the image.  

A lot of what we're interested in in geology is often defined by small local changes in our input data.  We're often using local "texture" as well as overall value/color to make a determination about what's present.  There's a wealth of different methods to investigate, but gradients are a good place to start.

### Calculate Slope and Hillshade

Let's start by going back to the same bathymetry data we were working with before.  Sure, it's kinda boring data, but it's easy to reason about, and doesn't require much specalized knowledge to interpret.

Let's look at the local image gradients (i.e. the partial derivates of the surface in x/y).  To be a full gradient, we'd need to include information about the cellsize, but we'll ignore it for now.  For just a minute, let's ignore units...

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import rasterio as rio

from context import data
from context import utils

# Let's load data from a geotiff using rasterio...
with rio.open(data.gebco.seamounts, 'r') as src:
    bathy = src.read(1)

# Simple differences in each direction
dy, dx = np.gradient(bathy)

# And let's compare the gradient to the image
fig, axes = plt.subplots(nrows=3, sharex=True, sharey=True)
axes[0].imshow(bathy, cmap='Blues_r', vmax=0).cmap.set_over('green') 
axes[1].imshow(dy, vmin=-100, vmax=100, cmap='coolwarm')
im = axes[2].imshow(dx, vmin=-100, vmax=100, cmap='coolwarm')

# Set up a single shared colorbar.
cax = fig.add_axes([0.9, 0.3, 0.02, 0.4])
cbar = fig.colorbar(im, cax=cax, label='Gradient')
cbar.ax.yaxis.set_label_position('left')

for label, ax in zip(['Data', 'DY', 'DX'], axes):
    ax.set(ylabel=label, xticks=[], yticks=[])
plt.show()

Okay, great. Rate of change in each direction.  By itself, that's not too interesting.  However, let's take a look at the magnitude of these gradients.  This is basically slope (though, again, we're ignoring units for now).

In [ ]:
# This is essentially slope with different units.
gradient_magnitude = np.hypot(dx, dy)

# And let's compare the gradient to the image
fig, axes = plt.subplots(nrows=2, sharex=True, sharey=True)
axes[0].imshow(bathy, cmap='Blues_r', vmax=0).cmap.set_over('green') 
im = axes[1].imshow(gradient_magnitude, vmin=0, vmax=200, cmap='gray_r')

cax = fig.add_axes([0.9, 0.3, 0.02, 0.4])
cbar = fig.colorbar(im, cax=cax, label='Gradient Magnitude')
cbar.ax.yaxis.set_label_position('left')

for ax in axes:
    ax.set(xticks=[], yticks=[])
plt.show()

Now let's go ahead and do a quick slope calculation just to be able to compare:

In [ ]:
# First we need to get the cellsize so that we know the "run" in "rise over run"
with rio.open(data.gebco.seamounts, 'r') as src:
    # Assumes cells are square and north-south aligned
    cellsize_deg = src.transform.a 

# This actually varies from north to south, but we'll ignore that for now
cellsize_m = 111.3 * 1000 * cellsize_deg

# And now a slope calculation - Basically inverse tangent of gradient magnitude
dy_m, dx_m = np.gradient(bathy, cellsize_m, cellsize_m)
slope = np.degrees(np.arctan(np.hypot(dy_m, dx_m)))

# Quick comparison plot...
fig, axes = plt.subplots(nrows=2, sharex=True, sharey=True)
im1 = axes[0].imshow(gradient_magnitude, vmin=0, vmax=200, cmap='gray_r')
im2 = axes[1].imshow(slope, vmin=0, vmax=30, cmap='viridis')
fig.colorbar(im1, ax=axes[0])
fig.colorbar(im2, ax=axes[1])
axes[0].set(xticks=[], yticks=[], ylabel='Gradient Magnitude')
axes[1].set(xticks=[], yticks=[], ylabel='Slope')
plt.show()

You might have noticed that the gradient magnitude image is displayed with a reversed gray colormap so that high values are black.  This is deliberate: It makes it visually similar to a hillshade, and it helps to treat the high gradient areas as shadows.  However, given that this is geological image processing, we'd be remiss not to talk about hillshade. It's a different filter, but one that's commonly applied to non-topographic data in geology.  It's a great way of visually highlighting small changes in an otherwise smoothly varying surface. As a result, those of us in the geosciences tend to use it as a visualization technique on all sorts of data.

Let's compare gradient magnitude and hillshade.  They're quite different, despite the visual similarity of the gradient magnitude visualization to shadows.  Hillshade highlights smaller features, but is therefore much noiser:  (Note plot below toggles layers -- note the controls below/on the plot. The default view will be the bathymetry data.)

In [ ]:
def hillshade(data, azdeg=315, altdeg=45, ve=1, cellsize=1):
    """Don't actually use this - Just showing that it's straightforward."""
    # Using trig here, but this is the dot product of the illumination vector 
    # with the normal vector of the surface at each point
    az = np.radians(90 - azdeg)
    alt = np.radians(altdeg)
    dy, dx = np.gradient(ve * -data, -cellsize, cellsize)
    aspect = np.arctan2(dy, dx)
    slope = 0.5 * np.pi - np.arctan(np.hypot(dx, dy))
    intensity = (np.sin(alt) * np.sin(slope) +
                 np.cos(alt) * np.cos(slope) * np.cos(az - aspect))
    return intensity

fig, ax = plt.subplots()
ax.imshow(bathy, cmap='Blues_r', vmax=0).cmap.set_over('green')
im1 = ax.imshow(gradient_magnitude, vmin=0, vmax=200, cmap='gray_r',
                label='Grad. Mag.')
im2 = ax.imshow(hillshade(bathy), vmin=-1, vmax=1, cmap='gray',
                label='Hillshade')

ax.set(xticks=[], yticks=[])
utils.Toggler(im1, im2).show()

Hillshade makes it difficult to see the long-wavelength / large-scale changes in the underlying data.  For that reason, it's most commonly combined with a colormapped version of the underlying data to produce a visualization that shows both large-scale changes and fine detail.  You've undoubtedly seen this a gazillion times, but it's very useful:

In [ ]:
from matplotlib.colors import LightSource

# We'll use a very slightly downsampled version to keep it fast
# That's the reason for bathy[::2, ::2]
# Also note that we're re-using "cellsize_m" from earlier...
ls = LightSource(azdeg=315, altdeg=45)
rgb = ls.shade(bathy[::2, ::2], cmap=plt.get_cmap('Blues_r'), 
               blend_mode='soft', dx=cellsize_m, dy=cellsize_m, 
               vert_exag=5)

fig, ax = plt.subplots(constrained_layout=True)
ax.imshow(rgb)
ax.set(xticks=[], yticks=[], title='Shaded Bathymetry')
plt.show()

### Edge Detection Filters

Okay, let's go back to our earlier gradient magnitude plot for a bit:

In [ ]:
fig, axes = plt.subplots(nrows=2, sharex=True, sharey=True)
axes[0].imshow(bathy, cmap='Blues_r', vmax=0).cmap.set_over('green') 
im = axes[1].imshow(gradient_magnitude, vmin=0, vmax=200, cmap='gray_r')

cax = fig.add_axes([0.9, 0.3, 0.02, 0.4])
cbar = fig.colorbar(im, cax=cax, label='Gradient Magnitude')
cbar.ax.yaxis.set_label_position('left')

for ax in axes:
    ax.set(xticks=[], yticks=[])

plt.show()

As you can see, the steep edges of each seamount are highlighted in black.

A term you'll hear frequently in image processing is ["edge detection"](https://en.wikipedia.org/wiki/Edge_detection).   The simplest and best known of these methods is the [Sobel filter](https://en.wikipedia.org/wiki/Sobel_operator).  It's almost exactly identical to the gradient calculations we've been using so far.  Let's take a closer look:

In [ ]:
# The near-equivalent of `np.gradient` is a "sobel filter" in image processing terms
import scipy.ndimage 

# The divide by 8 here is because the absolute value of the sobel kernel sums to 8.
# It's not _exactly_ the same as np.gradient, but it's _very_ close. Think of it as
# a unitless, more efficient np.gradient with a little more averaging.
correction = 8
sobel_dy = scipy.ndimage.sobel(bathy.astype(int), axis=0) / correction
sobel_dx = scipy.ndimage.sobel(bathy.astype(int), axis=1) / correction
sobel_grad_mag = scipy.ndimage.generic_gradient_magnitude(bathy.astype(int), 
                                                          scipy.ndimage.sobel) / correction

# ---------------------------------------------------------------------------------
# Now let's make a fancy figure that just shows that they're near-identical...
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(7, 7), sharex=True, sharey=True)

im1 = axes[0,0].imshow(dy, vmin=-100, vmax=100, cmap='coolwarm')
axes[1,0].imshow(dx, vmin=-100, vmax=100, cmap='coolwarm')
im2 = axes[2,0].imshow(gradient_magnitude, vmin=0, vmax=200, cmap='gray_r')

axes[0,1].imshow(sobel_dy, vmin=-100, vmax=100, cmap='coolwarm')
axes[1,1].imshow(sobel_dx, vmin=-100, vmax=100, cmap='coolwarm')
axes[2,1].imshow(sobel_grad_mag, vmin=0, vmax=200, cmap='gray_r')

for ax in axes.flat:
    ax.set(xticks=[], yticks=[])
axes[0,0].set(title='np.gradient')
axes[0,1].set(title='scipy.ndimage.sobel')

# Right hand colorbar
cax = fig.add_axes([0.85, 0.3, 0.02, 0.4])
cbar = fig.colorbar(im1, cax=cax, label='Gradient')
cbar.ax.yaxis.set_label_position('left')

# Left hand colorbar
cax = fig.add_axes([0.08, 0.3, 0.02, 0.4])
cbar = fig.colorbar(im2, cax=cax, label='Gradient Magnitude')
cbar.ax.yaxis.tick_left()
cbar.ax.yaxis.set_label_position('right')
cbar.ax.yaxis.label.set(rotation=-90, va='bottom')

fig.subplots_adjust(left=0.15, right=0.8)
plt.show()


They're identical for a lot of use cases other than one being 8x more than the other. However, note the slight differences even after we correct for the factor-of-8 difference between the two.

Remember filters and kernels?  This is a type of filter, and the difference is because `np.gradient` kernel in the x-direction is basically `[[-0.5, 0, 0.5]]`, and the sobel kernel in the x-direction is:
    
    [[-1, 0, 1],
     [-2, 0, 2],
     [-1, 0, 1]]
     
In practice, the only difference is scaling (factor of 8) and a bit more averaging over nearby pixels.  They're essentially the same thing.

Regardless, use `np.gradient` when you care about the real-world units (e.g. a slope calculation) and use as sobel filter (or other image processing edge filter) when you only care about relative differences.

Okay, we're spending a lot of time on gradients, but let's briefly cover one more.  For many images, it's useful to have a slightly smoother gradient.  You could blur the input image slightly and then take the gradient, but you can accomplish the same thing in fewer steps with a single filter.  A common smoothing/blurring filter is a guassian filter.  You use the derivative of a guassian function as a filter to calulate the effect of filtering with a guassian

In [ ]:
sigma = 5
gauss_dy = scipy.ndimage.gaussian_filter1d(bathy.astype(int), sigma, axis=0, order=1)
gauss_dx = scipy.ndimage.gaussian_filter1d(bathy.astype(int), sigma, axis=1, order=1)
gauss_grad_mag = scipy.ndimage.gaussian_gradient_magnitude(bathy.astype(int), sigma)

# ---------------------------------------------------------------------------------
# Now let's make a fancy figure that just shows that they're near-identical...
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(7, 7), sharex=True, sharey=True)

axes[0,0].imshow(dy, vmin=-100, vmax=100, cmap='coolwarm')
axes[1,0].imshow(dx, vmin=-100, vmax=100, cmap='coolwarm')
axes[2,0].imshow(gradient_magnitude, vmin=0, vmax=200, cmap='gray_r')

im_dy = axes[0,1].imshow(gauss_dy, vmin=-100, vmax=100, cmap='coolwarm')
im_dx = axes[1,1].imshow(gauss_dx, vmin=-100, vmax=100, cmap='coolwarm')
im_mag = axes[2,1].imshow(gauss_grad_mag, vmin=0, vmax=200, cmap='gray_r')

# Customize ticks and labels
for ax in axes.flat:
    ax.set(xticks=[], yticks=[])
axes[0,0].set(title='np.gradient')
axes[0,1].set(title='Gaussian')

# Right hand colorbar
cax = fig.add_axes([0.85, 0.3, 0.02, 0.4])
cbar = fig.colorbar(im_dx, cax=cax, label='Gradient')
cbar.ax.yaxis.set_label_position('left')

# Left hand colorbar
cax = fig.add_axes([0.08, 0.3, 0.02, 0.4])
cbar = fig.colorbar(im_mag, cax=cax, label='Gradient Magnitude')
cbar.ax.yaxis.tick_left()
cbar.ax.yaxis.set_label_position('right')
cbar.ax.yaxis.label.set(rotation=-90, va='bottom')

fig.subplots_adjust(left=0.15, right=0.8)

plt.show()


`sigma` is analagous to the standard deviation of a normal distribution's "bell curve".  The units are pixels, but the kernel in the filter is larger than `sigma`.  Regardless, increasing it leads to more smoothing.  To get a sense of how `sigma` varies, here's a quick interactive plot:

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)
im = ax.imshow(gauss_grad_mag, vmin=0, vmax=200, cmap='gray_r')
ax.set(xticks=[], yticks=[])

def update(sigma):
    if sigma == 0:
        grad = gradient_magnitude
    else:
        grad = scipy.ndimage.gaussian_gradient_magnitude(bathy.astype(float), sigma)
    im.set_data(grad)

# Note: This slider is fixed to integer intervals, but `sigma` doesn't have to be
utils.Slider(ax, 0, 20, update, start=0).show()

As you can see, this is great for identifying large scale changes.  Edge filters tend to enhance noise, and a gaussian gradient magnitude filter gives you a flexible way of supressing some noise.  It's often applied to photographics and other "non-smooth" image data for that reason.  We'll come back to it later, but for now, let's start to actually use these edge detection filters for something...

### Finding the Toe of Slope

We've spent _a lot_ of time on seamounts so far, but let's do one more example with them. Each seamount is surrounded by a talus cone that's much larger than the seamount itself. It's often useful to be able to identify the subtle end of these sort of features.  In common terms, we're looking for the toe of the slope.  However, the same idea shows up in many other applications. Similarly, we might want to find the base of the steep cliffs surrounding the seamounts, regardless of what exact depth it's at.

Gradients are first derivates.  What about second derivates?  Let's look at curvature.  We'll only explore one filter for curvature: A Gaussian Laplace filter.  First derivates are noisy, but second derivates are even more so.  As a result, the noise supression inherent in the gaussian derivatives we just talked about is _very_ useful in second derivative.

Why second derivates? They're good at finding maximum convexity or maximum concavity, which is what we're looking for in a "toe of slope" or "base of cliff" measurement.

In [ ]:
sigma = 5
gauss_laplace = scipy.ndimage.gaussian_laplace(bathy.astype(float), sigma)

fig, ax = plt.subplots()

im = ax.imshow(gauss_laplace, cmap='coolwarm', vmin=-10, vmax=10, label='Laplace')
ax.imshow(rgb, extent=im.get_extent(), zorder=-1) # Re-using our hillshaded plot earlier...

ax.set(xticks=[], yticks=[])
utils.Toggler(im).show()

In [ ]:
# Quickly re-run our seamount detection and buffer it by 5 pixels
background = scipy.ndimage.uniform_filter(bathy, 120)
seamounts = bathy > (background + 500)
seamounts = scipy.ndimage.binary_dilation(seamounts, iterations=5)

# We want only convex edges, not concave (Negative values are concave)
convexity_thresh = 2
convex = gauss_laplace > convexity_thresh
convex[seamounts] = False

fig, ax = plt.subplots(constrained_layout=True)
im = ax.imshow(np.ma.masked_equal(convex, 0), vmin=0, interpolation='nearest')
ax.imshow(rgb, extent=im.get_extent(), zorder=-1)
ax.set(xticks=[], yticks=[])
plt.show()

Now let's turn those regions into more discrete lines. 

To do this, we'll use a "skeltonization" operation (a.k.a. a medial transform).  This is also the first time we'll depart from `scipy.ndimage` and start diving into scikit image.  Up until now, all of these operations are N-dimensional and could be preformed on volumes as well as 2D images.  We'll start to depart from that now...

In [ ]:
import skimage.morphology

linear_toe_of_slope = skimage.morphology.skeletonize(convex)

fig, ax = plt.subplots(constrained_layout=True)
im = ax.imshow(np.ma.masked_equal(linear_toe_of_slope, 0), 
               vmin=0, interpolation='nearest')
ax.imshow(rgb, extent=im.get_extent(), zorder=-1)
ax.set(xticks=[], yticks=[])
plt.show()

Hey, not bad! It can definitely be improved upon, but for a few minutes work, we've done a relatively okay job at defining something that's actually fairly tricky to get right.

### Take Home Challenge

Let's keep this one simple and hard all at the same time... **Can you tweak the parameters/methods we used to find toe of slope to do a better job?**  What parameters would you try adjusting first?   Can you get a more continuous result?  Can you exclude false positives away from seamounts?

### Next Section

We'll look at lineaments and grain orientation analysis next:
[03 - Orientation Analysis](./03%20-%20Orientation%20Analysis.ipynb)